In [1]:
import os
import math
import numpy as np
import cfgrib
import pandas as pd
import xarray as xr
import fortranformat as ff
from glob import glob
# from astropy.io import ascii
# from astropy.table import Table

In [2]:
infolder_list = [r'G:\Modeling\223130\HRRR\02.03.2023.noaa.sg',
                 r'G:\Modeling\223130\HRRR\02.04.2023.noaa.sg',
                 r'G:\Modeling\223130\HRRR\02.05.2023.noaa.sg',
                 r'G:\Modeling\223130\HRRR\02.06.2023.noaa.sg',
                 r'G:\Modeling\223130\HRRR\02.07.2023.noaa.sg',
                 r'G:\Modeling\223130\HRRR\02.08.2023.noaa.sg']

out_asciifolder = r'P:\CMaderia\test'  #'G:\Modeling\223130\HRRR\ascii_output'
out_logfolder = r'P:\CMaderia\test\log'  #r'G:\Modeling\223130\HRRR\ascii_output\log'

In [3]:
surface_vars = ['MYR', 'MMO', 'MDAY', 'MHR', 'IX', 'JX', 'PRES', 'RAIN', 'SC', 'RADSW', 'RADLW', 'T2', 'Q2', 'WD10', 'WS10', 'SST']
vertical_vars = ['PRES', 'Z', 'TEMPK', 'WD', 'WS', 'W', 'RH', 'VAPMR', 'CLDMR', 'RAINMR', 'ICEMR', 'SNOWMR', 'GRPMR'] 
# 'ICEMR' not available in data

Steps:
1) Formulas
2) Unit conversions
3) Rename columns
4) Formatting (float/int decimal places)
5) Combine data frames
6) Export as CSV
7) Export as ASCII


# Functions

#### General

In [4]:
# Create a new pandas dataframe from an xarray input
def create_df(dataset, fieldnames:list):
    dataset = dataset.get(fieldnames)
    return dataset.to_dataframe()

In [5]:
# Calculate wind speed (dataframe)
def calc_windspeed(dataframe, ufield:str, vfield:str):
    magnitude = np.sqrt(np.square(dataframe[ufield]) + np.square(dataframe[vfield]))
    
    return magnitude

In [6]:
# Calculate wind direction (dataframe)
def calc_winddir(dataframe, longfield:str, ufield:str, vfield:str):
    angle2 = 0.622515*((dataframe[longfield]-360)+97.5)*0.017453
    
    # calculate U and V components
    Un = np.cos(angle2) * dataframe[ufield] + np.sin(angle2) * dataframe[vfield]
    Vn = (-1) * np.sin(angle2) * dataframe[ufield] + np.cos(angle2) * dataframe[vfield]
    # wind direction (degrees)
    angle = (270 - np.arctan2(Vn, Un) * 180 / math.pi)
    
    return angle

In [7]:
# Parse out day, month, year
def parse_date(gribname, dataframe, datefield:str):
    dataframe['MYR'] = dataframe[datefield].astype(str).str.split('-').str[0]
    dataframe['MMO'] = dataframe[datefield].astype(str).str.split('-').str[1]
    dataframe['MDAY'] = dataframe[datefield].astype(str).str.split('-').str[2].str.split(' ').str[0]
    dataframe['MHR'] = gribname.split('.')[1][1:3]  # take this one from the filename b/c it's not always in the date field
    
    return dataframe

In [8]:
# change field type
def change_field_type(dataframe, field_names:list, new_fieldtype:str):
    #int, float, bool, str/object
    for f in field_names:
        if new_fieldtype in ['int', 'int32', 'int64', 'float', 'float32', 'float64']:
            # convert to float first, then round up
            dataframe[f] = dataframe[f].astype(float)
            dataframe[f] = dataframe[f].round()
            # then convert to new field type
            dataframe[f] = dataframe[f].astype(new_fieldtype)
        else:
            # convert to new field type
            dataframe[f] = dataframe[f].astype(new_fieldtype)
    return dataframe

In [9]:
# given two data frames, removes duplicate columns between them
# src: https://www.geeksforgeeks.org/prevent-duplicated-columns-when-joining-two-pandas-dataframes/
def remove_dup_cols(df_removedups, df_main, cols_to_keep:list):
    # df_main = retains all columns
    # df_removedups = retains only unique columns (between the two data frames)
    # cols_to_keep = any duplicate columns that should be kept (i.e., for merging)
    # returns second data frames with duplicate columns removed
        different_cols = df_removedups.columns.difference(df_main.columns)
        df_removedups = df_removedups[list(different_cols) + cols_to_keep]
        return df_removedups

#### vertical

In [10]:
# do HRRR surface conversions
def calc_RH(dataframe, specific_humidity:str, pressure:str, temp:str):
    # Pressure in Pa, height in gpm, temp in K
    rh = (dataframe[specific_humidity]) / (0.622 * 6.112/dataframe[pressure] * \
                                                      np.exp(17.67 * (dataframe[temp]-273.15)/(dataframe[temp]-29.65)))
    return rh

#### surface

In [11]:
# do HRRR surface conversions
def calc_slpressure(dataframe, pressure:str, height:str, temp:str):
    # Pressure in Pa, height in gpm, temp in K
    pres = (dataframe[pressure] * \
                         np.power((1 - 0.0065*dataframe[height]/(dataframe[temp]+0.0065*dataframe[height])), -5.257)) / 100
    return pres

def calc_rainfall(dataframe, rain:str):  
    # RAIN  = prate (kg/m2/s to cm)
    rainfall = dataframe[rain] * 60 * 60 * 0.1
    return rainfall

def calc_snowcover(dataframe, snow:str):
    # SC - 50+ % = 1
    dataframe.loc[dataframe[snow]<50.0, snow] = 0
    dataframe.loc[dataframe[snow]>=50.0, snow] = 1
    sc = dataframe[snow]
    return sc

# no conversion needed for these
# RADSW = dswrf
# RADLW = ulwrf

#### run functions

In [12]:
# VERTICAL
# in = path of grib file, out = Fortran-formatted dataframe
def process_hybrid(grib_in, log_out):
    ds = xr.open_dataset(grib_in, engine='cfgrib', \
                         backend_kwargs={'filter_by_keys': {'typeOfLevel': 'hybrid'}})
    
    df_hybrid = create_df(ds, ['pres', 'gh', 't', 'u', 'v', 'w', 'q', 'clwmr', 'rwmr', 'snmr', 'grle'])
    
    df_hybrid = df_hybrid.reset_index()  # so x, y show up

    # calculate relative humidity first
    df_hybrid['RH'] = calc_RH(df_hybrid, 'q', 'pres', 't')

    # add 10 to x (i) and y (j)
    df_hybrid['x'] = df_hybrid['x'] + 10
    df_hybrid['y'] = df_hybrid['y'] + 10

    # do HRRR hybrid level conversions
    df_hybrid['PRES'] = df_hybrid['pres'] / 100  # Pa to hPa (mb)
    #df_hybrid.drop('pres', axis=1, inplace=True)
    df_hybrid['WD'] = calc_winddir(df_hybrid, 'longitude', 'u', 'v')
    df_hybrid['WS'] = calc_windspeed(df_hybrid, 'u', 'v')
    df_hybrid['q'] = df_hybrid['q'] * 1000
    df_hybrid['CLDMR'] = df_hybrid['clwmr'] * 1000
    df_hybrid['RAINMR'] = df_hybrid['rwmr'] * 1000
    df_hybrid['ICEMR'] = 0.0
    df_hybrid['SNOWMR'] = df_hybrid['snmr'] * 1000
    df_hybrid['GRPMR'] = df_hybrid['grle'] * 1000

    df_hybrid = parse_date(os.path.basename(grib_in), df_hybrid, 'valid_time')

    
    # FINAL FORMATTING
    vertical_layers_df = df_hybrid.copy()
    # rename columns
    vertical_layers_df.rename(columns={'gh':'Z', 't':'TEMPK', 'w': 'W', 'q': 'VAPMR', 'x':'IX', 'y':'JX', 'hybrid':'LEVEL'}, inplace=True)

    # change field types
    vertical_layers_df = change_field_type(vertical_layers_df, ['LEVEL', 'MYR', 'MMO', 'MDAY', 'MHR', 'IX', 'JX', \
                                                                'PRES', 'Z', 'WD', 'RH'], 'int')
    # export ascii
    vertical_layers_ascii = vertical_layers_df[vertical_vars]

    # export csv (for QC)
    vertical_vars_csv = vertical_vars + ['IX', 'JX', 'VAPMR', 'pres', 'TEMPK', 'u', 'v', 'latitude', 'longitude']
    vertical_layers_csv = vertical_layers_df.loc[:, vertical_vars_csv]

    # * = used for calculations
    vertical_layers_csv.rename(columns={'pres':'p*', 'VAPMR':'qq*', 'TEMPK': 'tk*', 'u': 'u*', \
                                       'v':'v*', 'valid_time':'valid_time*'}, inplace=True)

    # EXPORT CSV
    # write log file (CSV)
    vertical_layers_csv.to_csv(log_out, index=False)


    # FORTRAN FORMATTING
    # Fortran formatting for each column
    vertical_layers_ascii = vertical_layers_ascii.round({'PRES': 1})
    vertical_layers_ascii = vertical_layers_ascii.round({'RAIN': 2})
    vertical_layers_ascii = vertical_layers_ascii.round({'RADSW': 1})
    vertical_layers_ascii = vertical_layers_ascii.round({'RADLW': 1})
    vertical_layers_ascii = vertical_layers_ascii.round({'T2': 1})
    vertical_layers_ascii = vertical_layers_ascii.round({'Q2': 2})
    vertical_layers_ascii = vertical_layers_ascii.round({'WD10': 1})
    vertical_layers_ascii = vertical_layers_ascii.round({'WS10': 1})
    vertical_layers_ascii = vertical_layers_ascii.round({'SST': 1})

    # apply formatting to each pandas dataframe column
    # https://stackoverflow.com/questions/30904333/write-pandas-dataframe-to-file-using-fortran-format-string
    format_string = '(i4, i6, f6.1, i4, f5.1, f6.2, i3, f5.2, f6.3, f6.3, f6.3, f6.3, f6.3)'
    header_line = ff.FortranRecordWriter(format_string)
    vertical_layers_ascii = vertical_layers_ascii.apply(lambda x : header_line.write(x.values),axis=1)
    
    return vertical_layers_ascii

In [13]:
# SURFACE / TOP
# in = path of grib file, out = Fortran-formatted dataframe
def process_surface(grib_in, log_out):
    
    # SURFACE
    ds = xr.open_dataset(grib_in, engine='cfgrib', \
                         backend_kwargs={'filter_by_keys': {'stepType': 'instant', 'typeOfLevel': 'surface'}})
    # for v in ds:
    #     print("{}, {}, {}".format(v, ds[v].attrs['long_name'], ds[v].attrs['units']))

    df_surface = create_df(ds, ['sp', 'orog', 'prate', 'snowc', 'dswrf'])
    
    df_surface = df_surface.reset_index()  # so x, y show up

    # add 10 to x (i) and y (j)
    df_surface['x'] = df_surface['x'] + 10
    df_surface['y'] = df_surface['y'] + 10

    # do HRRR surface conversions
    #df_surface['PRES'] = calc_slpressure(df_surface, 'sp', 'orog', 't')
    df_surface['RAIN'] = calc_rainfall(df_surface, 'prate')
    df_surface['SC'] = calc_snowcover(df_surface, 'snowc')
    
    df_surface = parse_date(os.path.basename(grib_in), df_surface, 'valid_time')
    
    
    # HEIGHT ABOVE GROUND - 2 METERS
    ds = xr.open_dataset(grib_in, engine='cfgrib', \
                         backend_kwargs={'filter_by_keys': {'level': 2, 'typeOfLevel': 'heightAboveGround'}})

    df_hag2 = create_df(ds, ['t2m', 'sh2', 'r2'])
    
    df_hag2 = df_hag2.reset_index()

    # add 10 to x (i) and y (j)
    df_hag2['x'] = df_hag2['x'] + 10
    df_hag2['y'] = df_hag2['y'] + 10

    # do HRRR 2 m conversions
    # T2 = t2m (K, no conversion needed)
    # Q2 = sh2 * 1000 (g/kg, no conversion needed)
    df_hag2['sh2'] = df_hag2['sh2'] * 1000
    # R2?
    # https://carnotcycle.wordpress.com/2012/08/04/how-to-convert-relative-humidity-to-absolute-humidity/
    # df_hag2['R2'] = (6.112 * np.exp((17.67 * (df_hag2['t2m']-273.15))/(df_hag2['t2m']-29.65)) * df_hag2['r2'] * 2.1674) / df_hag2['t2m']

    df_hag2 = parse_date(os.path.basename(grib_in), df_hag2, 'valid_time')
    
    
    # HEIGHT ABOVE GROUND - 10 METERS
    ds = xr.open_dataset(grib_in, engine='cfgrib', \
                         backend_kwargs={'filter_by_keys': {'level': 10, 'typeOfLevel': 'heightAboveGround'}})

    df_hag10 = create_df(ds, ['u10', 'v10', 'si10'])
    
    df_hag10 = df_hag10.reset_index()

    # add 10 to x (i) and y (j)
    df_hag10['x'] = df_hag10['x'] + 10
    df_hag10['y'] = df_hag10['y'] + 10

    # do HRRR 10 m conversions
    df_hag10['WD10'] = calc_winddir(df_hag10, 'longitude', 'u10', 'v10')
    df_hag10['WS10'] = calc_windspeed(df_hag10, 'u10', 'v10')
    df_hag10['SST'] = 0.0

    df_hag10 = parse_date(os.path.basename(grib_in), df_hag10, 'valid_time')
    
    
    # NOMINAL TOP
    ds = xr.open_dataset(grib_in, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'nominalTop'}})

    df_top = create_df(ds, ['ulwrf'])

    df_top = df_top.reset_index()

    # add 10 to x (i) and y (j)
    df_top['x'] = df_top['x'] + 10
    df_top['y'] = df_top['y'] + 10

    df_top = parse_date(os.path.basename(grib_in), df_top, 'valid_time')

    
    # JOIN THE SURFACE / TOP LAYERS
    # remove duplicate columns (between data frames) - do this before merge
    df_hag2 = remove_dup_cols(df_hag2, df_surface, ['y', 'x'])
    df_hag10 = remove_dup_cols(df_hag10, df_surface, ['y', 'x'])
    df_top = remove_dup_cols(df_top, df_surface, ['y', 'x'])

    # do a triple merge (to join all surface / top layers on (IX, JX))
    surface_layers_df = pd.merge(df_surface, pd.merge(df_top, pd.merge(df_hag2, df_hag10, on=['y', 'x']), on=['y', 'x']), on=['y', 'x'])
    
    
    # FINAL CALCULATION
    # do HRRR surface conversions
    surface_layers_df['PRES'] = calc_slpressure(surface_layers_df, 'sp', 'orog', 't2m')
    
    
    # FINAL FORMATTING
    # drop duplicate fields (from merge)
    surface_layers_df.drop([i for i in list(surface_layers_df.columns) if '_x' in i or '_y' in i], axis=1, inplace=True)
    # rename columns
    surface_layers_df.rename(columns={'dswrf':'RADSW', 'ulwrf':'RADLW', 't2m': 'T2', 'sh2': 'Q2', 'r2':'R2', 'x':'IX', 'y':'JX'}, inplace=True)
    
    
    surface_layers_df['SC'] = surface_layers_df['SC'].round()
    # change field types
    surface_layers_df = change_field_type(surface_layers_df, ['MYR', 'MMO', 'MDAY', 'MHR', 'IX', 'JX', 'SC'], 'int')

    # export ascii
    surface_layers_ascii = surface_layers_df[surface_vars]

    # export csv (for QC)
    surface_vars_csv = surface_vars + ['sp', 'orog', 'u10', 'v10', 'prate', 'snowc', 'latitude', 'longitude', 'valid_time']
    surface_layers_csv = surface_layers_df.loc[:, surface_vars_csv]

    # * = used for calculations
    surface_layers_csv.rename(columns={'sp':'P*', 'orog':'h*', 'T2': 'T2*', 'u10':'u10*', 'v10':'v10*', 'prate': 'prate_RAIN*', \
                                       'snowc':'snowc_SC*', 'valid_time':'valid_time*'}, inplace=True)
    
    # write log file (CSV)
    surface_layers_csv.to_csv(log_out, index=False)
    
    
    # FORTRAN FORMATTING
    # Fortran formatting for each column
    surface_layers_ascii = surface_layers_ascii.round({'PRES': 1})
    surface_layers_ascii = surface_layers_ascii.round({'RAIN': 2})
    surface_layers_ascii = surface_layers_ascii.round({'RADSW': 1})
    surface_layers_ascii = surface_layers_ascii.round({'RADLW': 1})
    surface_layers_ascii = surface_layers_ascii.round({'T2': 1})
    surface_layers_ascii = surface_layers_ascii.round({'Q2': 2})
    surface_layers_ascii = surface_layers_ascii.round({'WD10': 1})
    surface_layers_ascii = surface_layers_ascii.round({'WS10': 1})
    surface_layers_ascii = surface_layers_ascii.round({'SST': 1})

    # apply formatting to each pandas dataframe column
    # https://stackoverflow.com/questions/30904333/write-pandas-dataframe-to-file-using-fortran-format-string
    format_string = '(i4, i2, i2, i2, i3, i3, f7.1, f5.2, i2, f8.1, f8.1, f8.1, f8.2, f8.1, f8.1, f8.1)'
    header_line = ff.FortranRecordWriter(format_string)
    surface_layers_ascii = surface_layers_ascii.apply(lambda x : header_line.write(x.values),axis=1)
    
    return surface_layers_ascii

In [14]:
# export final combined ASCII file!
# in = output file path, input vertical layers, input surface layers
# out = final combined ascii file (in Fortran format)
def export_ascii(output_file, in_vertical, in_surface):
    with open(output_file, 'w') as outfile:
        count = 0
        for row in in_surface:
            outfile.write(row+'\n')

            # there are 3025 x, y pairs, and 50 levels for each x, y pair - loop through them using a range
            for i in range(count+0, 3025*50, 3025):
                row = in_vertical[i]
                outfile.write(row+'\n')
            count = count + 1

# LOOP

In [ ]:
for infolder in infolder_list:
    grib_list = glob(os.path.join(infolder, '*.grib2'))
    for g in grib_list:
        
        # DIRECTORY SET UP
        out_asciifolderpath = os.path.join(out_asciifolder, os.path.basename(infolder))
        # make the log folder directory to store the output for each individual grib run, if it does not already exist
        if os.path.exists(out_asciifolderpath) == False: os.mkdir(out_asciifolderpath)
        
        out_logfolderpath = os.path.join(out_asciifolderpath, 'log')
        # make the log folder directory to store the output for each individual grib run, if it does not already exist
        if os.path.exists(out_logfolderpath) == False: os.mkdir(out_logfolderpath)
        
        # FILE EXPORT
        # process vertical layers first, format them, and export log
        out_verticallog = os.path.join(out_logfolderpath, os.path.basename(g).replace('.grib2', '') + '_vertical_data' + '.csv')
        #print(out_verticallog)
        vertical = process_hybrid(g, out_verticallog)
        
        # process surface layers next, format them, and export log
        out_surfacelog = os.path.join(out_logfolderpath, os.path.basename(g).replace('.grib2', '') + '_surface_data' + '.csv')
        #print(out_surfacelog)
        surface = process_surface(g, out_surfacelog)
        
        # combine vertical and surface, and export Fortran-formatted ascii
        out_ascii_combined = os.path.join(out_asciifolderpath, os.path.basename(g)).replace('.grib2', '.dat')
        print(out_ascii_combined)
        export_ascii(out_ascii_combined, vertical, surface)

P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t00z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t01z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t02z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t03z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t04z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t05z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t06z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t07z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t08z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t09z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t10z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t11z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t12z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t13z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t14z.wrfnatf00.sg.dat
P:\CMaderia\test\02.03.2023.noaa.sg\hrrr.t15z.wrfnatf00